# Taylor-transformed skeletons

## 1. Taylor videos on skeletons

In [1]:
import scipy
from scipy import io
import torch

In [2]:
def factorial(n):
    """
    calculate factorial of each element in the input tensor using recursion.
    """
    # base case: factorial of 0 is 1
    if n == 0:
        return torch.tensor(1)
    else:
        return n * factorial(n - 1)

In [3]:
def taylor_video(videoinput, terms, temporal_block):
    
    # assume stride = 1
    video = torch.from_numpy(scipy.io.loadmat(videoinput)['skeletonsequence'])
    
    J, C, T = video.shape
    
    if temporal_block - 1 < terms:
        print('The given temporal block length is not enough to compute terms defined.')
        
    else:
        Taylor = torch.zeros((J, C, T - temporal_block + 1))
        
        for i in range(T - temporal_block + 1):
            # index starts from 0
            video_clip = video[:, :, i:i+temporal_block]
            slice_tensor = video[:, :, i].unsqueeze(2)
            dummy_clip = slice_tensor.repeat(1, 1, temporal_block)
            
            delta_temp = video_clip - dummy_clip
            
            D_temp = torch.zeros(J, C, terms)
            
            temp = video_clip
            
            for j in range(terms):
                diff = temp[:, :, 1:] - temp[:, :, :-1]
                D_temp[:, :, j] = diff[:, :, 0]
                temp = diff
            
            M = torch.zeros((J, C, temporal_block))
            
            for order in range(terms):
                M = M + (D_temp[:, :, order].unsqueeze(-1) / factorial(order)) * torch.pow(delta_temp, order)               
                
            taylor_frame = M.mean(2)

            Taylor[:, :, i] = taylor_frame
    
    return Taylor

In [4]:
sk = 'a01_s01_e01_skeleton3D.txtssq.mat'

# subsequence length = 4, terms = 3, displacement only
taylorskeleton = taylor_video(sk, 3, 4) 

In [5]:
skeleton = scipy.io.loadmat(sk)['skeletonsequence']
print(skeleton.shape)

skeleton = torch.from_numpy(skeleton)

(20, 3, 54)


In [6]:
taylorskeleton.shape

torch.Size([20, 3, 51])

## 2. Visualisation of original skeletons

In [7]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML, display

num_frames = skeleton.shape[2]
num_joints = skeleton.shape[0]  

# connections of bones
sk_config = [[12, 10, 8, 1, 2, 0, 7, 9, 11], [19, 2, 3, 6, 5, 14, 16, 18], [6, 4, 13, 15, 17]]

# plot 2D skeleton frame with connections 
# showing x as horizontal, y as vertical
def plot_skeleton_frame_2d(frame_data, sk_config, ax):
    ax.clear()
    ax.set_title("2D Skeleton Frame (Horizontal X, Vertical Y)")

    # plot joints: x as horizontal, y as vertical
    for i in range(num_joints):
        x = frame_data[i, 0]
        y = frame_data[i, 1]
        ax.scatter(x, y, color='r')  
    
    # connections
    for connections in sk_config:
        for i in range(len(connections) - 1):
            start_joint = connections[i]
            end_joint = connections[i + 1]
            ax.plot([frame_data[start_joint, 0], frame_data[end_joint, 0]],
                    [frame_data[start_joint, 1], frame_data[end_joint, 1]], color='b')  
    
    # equal aspect ratio
    ax.set_aspect('equal')  

fig, ax = plt.subplots(figsize=(8, 8))

# initialize the plot with the first frame
plot_skeleton_frame_2d(skeleton[:, :, 0], sk_config, ax)

# update the plot for each frame
def update(frame):
    plot_skeleton_frame_2d(skeleton[:, :, frame], sk_config, ax)

# animation
ani = FuncAnimation(fig, update, frames=num_frames, interval=100)  # Interval in milliseconds

# disable xticks, yticks, xlabel, and ylabel
ax.set_xticks([])
ax.set_yticks([])
ax.set_xlabel('')
ax.set_ylabel('')

# display the animation using HTML display
html_anim = HTML(ani.to_jshtml())
display(html_anim)

# hide the ticks and axis labels
plt.close()  

## 3. Visualisation of Taylor-transformed skeletons on original skeletons

In [8]:
num_taylor = taylorskeleton.shape[2]
num_frames = skeleton.shape[2]
num_joints = skeleton.shape[0]

# connections of bones
sk_config = [[12, 10, 8, 1, 2, 0, 7, 9, 11], [19, 2, 3, 6, 5, 14, 16, 18], [6, 4, 13, 15, 17]]

# plot 2D skeleton frame with connections 
# showing x as horizontal, y as vertical
def plot_skeleton_frame_2d(frame_data, taylor_data, sk_config, ax):
    ax.clear()
    ax.set_title("2D Skeleton Frame (Horizontal X, Vertical Y)")

    # plot joints: x as horizontal, y as vertical
    for i in range(num_joints):
        x = frame_data[i, 0]
        y = frame_data[i, 1]
        a = taylor_data[i, 0]
        b = taylor_data[i, 1]
        # adjust the scaling factor as needed
        ss = int(5000 * (abs(a) + abs(b)) / 2)  
        ax.scatter(x, y, color='r', s=ss)

    # connections
    for connections in sk_config:
        for i in range(len(connections) - 1):
            start_joint = connections[i]
            end_joint = connections[i + 1]
            ax.plot([frame_data[start_joint, 0], frame_data[end_joint, 0]],
                    [frame_data[start_joint, 1], frame_data[end_joint, 1]], color='b')  

    # equal aspect ratio
    ax.set_aspect('equal')

fig, ax = plt.subplots(figsize=(8, 8))

plot_skeleton_frame_2d(skeleton[:, :, 0], taylorskeleton[:, :, 0], sk_config, ax)

# update the plot for each frame
def update(frame):
    plot_skeleton_frame_2d(skeleton[:, :, frame], taylorskeleton[:, :, frame], sk_config, ax)

# animation
ani = FuncAnimation(fig, update, frames=num_taylor, interval=100)  # Interval in milliseconds

html_anim = HTML(ani.to_jshtml())
display(html_anim)

plt.close()


## 4. Compute Taylor videos on skeleton sequences

In [9]:
# import os

# # directory path
# directory = 'skeleton'

# # loop through files in the directory
# for filename in os.listdir(directory):
#     # check if the file ends with '.mat'
#     if filename.endswith('.mat'):
#         # full path of the file
#         filepath = os.path.join(directory, filename)
#         # process the file or perform operations as needed
#         taylorskeleton = taylor_video(filepath, 1, 4)
#         # new filename with 'taylor' added before '.mat'
#         new_filename = filename.replace('.mat', 'taylor.mat')
#         new_filepath = os.path.join(directory, new_filename)
#         # print(new_filepath)
#         print(f'Processing file: {filepath}')
#         taylorskeleton_array = taylorskeleton.double().numpy()

#         # save the numpy array to a matlab file with key 'skeletonsequence'
#         io.savemat(new_filepath, {'skeletonsequence': taylorskeleton_array}, appendmat=False, format='5')